In [ ]:
%load_ext aiida
%aiida

In [ ]:

from aiida.orm import StructureData
from aiida.engine import CalcJob


import ipywidgets as ipw
from IPython.display import display, clear_output
import common

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Delete all SPM extras:

#qb = QueryBuilder()
#qb.append(StructureData)
#
#for struct_data in qb.all():
#    extras = struct_data[0].extras
#    for key in extras.keys():
#        if key.endswith('_pk') and key.split('_')[1].isdigit():
#            struct_data[0].delete_extra(key)

In [ ]:
# Include:
# * list all spm
# * disable
# * view
# * invoke preprocess

In [ ]:
# 1st preprocess everything
common.preprocess_spm_calcs()

In [ ]:
button_node_map = {}

def toggle_obsolete(b):
    node, status, structure = button_node_map[b.model_id]
    is_obs = node.extras['obsolete'] if 'obsolete' in node.extras else False
    node.set_extra('obsolete', not is_obs)
    if is_obs:
        status.value = 'Success'
        b.description = 'Disable'
    else:
        status.value = 'OBSOLETE'
        b.description = 'Enable'
        # remove from input structure extras
        for key, val in structure.extras.items():
            if val == node.pk:
                structure.delete_extra(key)
        # remove preprocess extras
        if 'preprocess_error' in node.extras:
            node.delete_extra('preprocess_error')
        if 'preprocess_successful' in node.extras:
            node.delete_extra('preprocess_successful')
        if 'preprocess_version' in node.extras:
            node.delete_extra('preprocess_version')
        

table_vbox = ipw.VBox()

def populate_table():

    headers_and_colwidths = [
        ('PK', '50px'),
        ('Type', '60px'),
        ('Creation Time', '120px'),
        ('Structure', '200px'),
        ('Codes', '180px'),
        ('Status', '200px'),
        ('Manage', '80px')
    ]

    html_h_and_cw = [(ipw.HTML('<b>%s</b>' % h_cw[0]), h_cw[1]) for h_cw in headers_and_colwidths]
    boxed_headers = [ipw.HBox([h_cw[0]], layout=ipw.Layout(border='0.1px solid', width=h_cw[1])) for h_cw in html_h_and_cw]
    hboxes = [ipw.HBox(boxed_headers)]

    qb = QueryBuilder()
    qb.append(WorkChainNode, filters={
        'attributes.process_label': {'in': list(common.workchain_preproc_and_viewer_info.keys())}
    })
    qb.order_by({WorkChainNode:{'ctime':'desc'}})

    for i, node_tuple in enumerate(qb.iterall()):
        node = node_tuple[0]
        if not node.is_sealed:
            print("Skipping underway workchain PK %d"%node.pk)
            continue
        try:
            node_inp_dict = {e: node.inputs[e] for e in node.inputs}
            if "structure" in node.inputs:
                structure = node.inputs.structure
            if "slabsys_structure" in node.inputs:
                structure = node.inputs.slabsys_structure

            struct_pk = structure.pk
            formula = structure.extras['formula']
            wc_name = node.attributes['process_label'] 
            calc_type = wc_name[:-9] # cut off WorkChain
            
            ver = 0
            if 'version' in node.extras:
                ver = node.extras['version']
            
            code_labels = sorted([val.label for key, val in node_inp_dict.items() if "code" in key])
            code_str = " ".join(code_labels)

            status = "Success"
            disable_toggle = ipw.Button(description='Disable', layout=ipw.Layout(width='75px'))
            disable_toggle.on_click(lambda b: toggle_obsolete(b))
            
            ### ---------------------------------------------------------------
            ### View link
            view_link = ipw.HTML('')
            
            viewer_path = "../"+common.workchain_preproc_and_viewer_info[wc_name][ver]["viewer_path"]
            view_link.value = "<a target='_blank' href='%s?pk=%s'>View</a><br />" % (viewer_path, node.pk)
            ### ---------------------------------------------------------------

            if 'preprocess_error' in node.extras:
                status = "Preproc. error: %s" % node.extras['preprocess_error']
                disable_toggle = ipw.HTML('')
                view_link.value = ''
            
            is_obs = node.extras['obsolete'] if 'obsolete' in node.extras else False
            if is_obs:
                status = "OBSOLETE"
                disable_toggle.description = 'Enable'

            status_html = ipw.HTML("<p style='line-height:1.5'>%s</p>" % status)
            
            button_node_map[disable_toggle.model_id] = (node, status_html, structure)
            
            manage_section = ipw.VBox([disable_toggle, view_link])

            row_content = [
                ipw.HTML("<p style='line-height:1.5'>%s</p>" % node.pk),
                ipw.HTML("<p style='line-height:1.5'>%s</p>" % calc_type),
                ipw.HTML("<p style='line-height:1.5'>%s</p>" % node.ctime.strftime("%Y-%m-%d %H:%M")),
                ipw.HTML("<p style='line-height:1.5'>PK%d: %s</p>" % (struct_pk, formula)),
                ipw.HTML("<p style='line-height:1.5'>%s</p>" % code_str),
                status_html,
                manage_section
            ]

            boxed_row = [ipw.HBox([row_el], layout=ipw.Layout(border='0.1px solid', width=h_cw[1]))
                         for row_el, h_cw in zip(row_content, headers_and_colwidths)]

            hboxes.append(ipw.HBox(boxed_row))
        except Exception as e:
            raise(e)
            print(repr(e))
            print("Data retrieval failed for pk%d" % (node.pk))

    table_vbox.children += tuple(hboxes)

In [ ]:
populate_table()

In [ ]:
display(table_vbox)